In [1]:
import numbers
from typing import Dict

import numpy as np
%load_ext Cython

In [88]:
%%cython --annotate
import numbers
from typing import Dict

import numpy as np
cimport numpy as np
from libc.math cimport exp, log
from libc.stdlib cimport rand, RAND_MAX


cpdef transition(long[:,:] state, int t, theta, int n_particles, consts):
    cdef double c1 = exp(theta['lc1'])
    cdef double c2 = exp(theta['lc2'])
    cdef double c3 = exp(theta['lc3'])
    cdef double c4 = exp(theta['lc4'])
    cdef double c5 = consts['c5']
    cdef double c6 = consts['c6']
    cdef double c7 = exp(theta['lc7'])
    cdef double c8 = exp(theta['lc8'])
    cdef int k = consts['k']
    cdef long[:,:] S = consts['S']
    cdef int i
    
    state = state.copy()
    state_new = np.zeros(shape=(4, n_particles), dtype=np.long)
    
    for i in range(n_particles):
        state_new[:, i] = _simulate_state(state[:, i], S,
                                          c1, c2, c3, c4, c5, c6, c7, c8, k)

    return state_new


cdef _simulate_state(long[:] state, long[:,:] S,
                    double c1, double c2, double c3, double c4,
                    double c5, double c6, double c7, double c8,
                    int k):
    cdef int T = 1
    cdef double t = 0.0
    cdef double h_sum = 0.0
    cdef double s1, s2, dt, cum_prob
    cdef double[:] h = np.empty(shape=8)
    cdef double[:] p = np.empty(shape=8)
    cdef int i, reaction_type
    cdef long rna, P, P2, dna

    while t < T:
        # Calculate the hazard function.
        rna = state[0]
        P = state[1]
        P2 = state[2]
        dna = state[3]
        
        h[0] = c1 * dna * P2
        h[1] = c2 * (k - dna)
        h[2] = c3 * dna
        h[3] = c4 * rna
        h[4] = c5 * P * (P - 1) / 2
        h[5] = c6 * P2
        h[6] = c7 * rna
        h[7] = c8 * P
        
        # Calculate the hazard function sum and reaction probabilities.
        h_sum = 0.0
        for i in range(8):
            h_sum += h[i]
            
        for i in range(8):
            p[i] = h[i] / h_sum

        # Calculate time delta.
        s1 = rand() / RAND_MAX
        dt = -log(s1) / h_sum

        # Select reaction type.
        s2 = rand() / RAND_MAX
        cum_prob = 0.0
        
        for i in range(8):
            cum_prob += p[i]
            
            if s2 <= cum_prob:
                reaction_type = i
                break

        # Perform the selected reaction.
        for i in range(4):
            state[i] += S[i, reaction_type]
            
        t += dt

    return state

In [103]:
%%time
n_particles = 100
random_state = check_random_state(1)

consts = {
    'k': 10,
    'm': 5,
    'c5': 0.1,
    'c6': 0.9,
    'observation_std': 2.0,
    'S': np.array([
        [0, 0, 1, 0, 0, 0, -1, 0],
        [0, 0, 0, 1, -2, 2, 0, -1],
        [-1, 1, 0, 0, 1, -1, 0, 0],
        [-1, 1, 0, 0, 0, 0, 0, 0]
    ])
}

theta = {
    'lc1': np.log(0.1),
    'lc2': np.log(0.7),
    'lc3': np.log(0.35),
    'lc4': np.log(0.2),
    'lc7': np.log(0.3),
    'lc8': np.log(0.1)
}

state = random_state.randint(low=1, high=9, size=(4, n_particles))
# print(state)

for i in range(100):
    state = transition(state=state, t=i, theta=theta,
                       n_particles=n_particles, consts=consts)
#     print(state)

Wall time: 1.3 s


In [9]:
%%cython -a
from libc.stdlib cimport rand, RAND_MAX

cdef int i
cdef double num
for i in range(10):
    num = rand() / RAND_MAX
    print(num)

0.5316629535813471
0.5711844233527634
0.6017639698477126
0.6071657460249641
0.1662343211157567
0.663045136875515
0.45078890346995454
0.3521225623340556
0.057039094210638755
0.6076845606860561
